In [2]:
import json
import urllib3
import config
import pandas as pd
import string

http = urllib3.PoolManager()
call_string = 'www.thecocktaildb.com/api/json/' + config.api_version + '/' + config.api_key + '/search.php?f=' 

df_list = []
for i in string.ascii_lowercase:
    try :
        response = http.request('GET', call_string + i)
        data = json.loads(response.data)
        single_df = pd.json_normalize(data['drinks'])
        df_list.append(single_df)
    except:
        pass

for i in range(10):
    try :
        response = http.request('GET', call_string + str(i))
        data = json.loads(response.data)
        single_df = pd.json_normalize(data['drinks'])
        df_list.append(single_df)
    except:
        pass


df = pd.concat(df_list)
df.shape

(636, 51)

In [3]:
df.to_excel('drinks.xlsx')
cols = df.columns.to_list()
ing_cols = []
meas_cols = []
rem_cols = []

for c in range(len(cols)):
    if 'strIngredient' in cols[c]:
        ing_cols.append(cols[c])
    if 'strMeasure' in cols[c]:
        meas_cols.append(cols[c])
    if 'strIngredient' not in cols[c] and 'strMeasure' not in cols[c]:
        rem_cols.append(cols[c])

ing_cols.append('idDrink')
meas_cols.append('idDrink')
print(ing_cols)
print(meas_cols)

['strIngredient1', 'strIngredient2', 'strIngredient3', 'strIngredient4', 'strIngredient5', 'strIngredient6', 'strIngredient7', 'strIngredient8', 'strIngredient9', 'strIngredient10', 'strIngredient11', 'strIngredient12', 'strIngredient13', 'strIngredient14', 'strIngredient15', 'idDrink']
['strMeasure1', 'strMeasure2', 'strMeasure3', 'strMeasure4', 'strMeasure5', 'strMeasure6', 'strMeasure7', 'strMeasure8', 'strMeasure9', 'strMeasure10', 'strMeasure11', 'strMeasure12', 'strMeasure13', 'strMeasure14', 'strMeasure15', 'idDrink']


In [4]:
df_ing = df[ing_cols].melt(id_vars = 'idDrink')
df_meas = df[meas_cols].melt(id_vars = 'idDrink')

# getting index to match ingredient to volume
df_ing['ing_id'] = df_ing['variable'].replace('(strIngredient)', '', regex=True)
df_meas['meas_id'] = df_meas['variable'].replace('(strMeasure)', '', regex=True)

# filtering out Nones
df_ing = df_ing[df_ing['value'].notnull()]
df_meas = df_meas[df_meas['value'].notnull()]

# ingredient merge
ing_meas = pd.merge(df_ing, df_meas, how = 'left', left_on = ['idDrink', 'ing_id'], right_on = ['idDrink', 'meas_id'])
ing_meas = ing_meas[['idDrink', 'ing_id', 'value_x', 'value_y']]

# final dataframe merge 
final_df = pd.merge(df[rem_cols], ing_meas, how = 'left', left_on = 'idDrink', right_on = 'idDrink')
final_df.sort_values('idDrink').head(50)

final_df.to_excel('drinks_rollout.xlsx')